# Домашнее задание 3. Парсинг, Git и тестирование на Python

**Цели задания:**

* Освоить базовые подходы к web-scraping с библиотеками `requests` и `BeautisulSoup`: навигация по страницам, извлечение HTML-элементов, парсинг.
* Научиться автоматизировать задачи с использованием библиотеки `schedule`.
* Попрактиковаться в использовании Git и оформлении проектов на GitHub.
* Написать и запустить простые юнит-тесты с использованием `pytest`.


В этом домашнем задании вы разработаете систему для автоматического сбора данных о книгах с сайта [Books to Scrape](http://books.toscrape.com). Нужно реализовать функции для парсинга всех страниц сайта, извлечения информации о книгах, автоматического ежедневного запуска задачи и сохранения результата.

Важной частью задания станет оформление проекта: вы создадите репозиторий на GitHub, оформите `README.md`, добавите артефакты (код, данные, отчеты) и напишете базовые тесты на `pytest`.



In [251]:
!40pip install -q schedule pytest # установка библиотек, если ещё не

"40pip" ­Ґ пў«пҐвбп ў­гваҐ­­Ґ© Ё«Ё ў­Ґи­Ґ©
Є®¬ ­¤®©, ЁбЇ®«­пҐ¬®© Їа®Ја ¬¬®© Ё«Ё Ї ЄҐв­л¬ д ©«®¬.


In [252]:
# Библиотеки, которые могут вам понадобиться
# При необходимости расширяйте список
import time
import requests
import schedule
from urllib.parse import urljoin, urlparse
from bs4 import BeautifulSoup

## Задание 1. Сбор данных об одной книге (20 баллов)

В этом задании мы начнем подготовку скрипта для парсинга информации о книгах со страниц каталога сайта [Books to Scrape](https://books.toscrape.com/).

Для начала реализуйте функцию `get_book_data`, которая будет получать данные о книге с одной страницы (например, с [этой](http://books.toscrape.com/catalogue/a-light-in-the-attic_1000/index.html)). Соберите всю информацию, включая название, цену, рейтинг, количество в наличии, описание и дополнительные характеристики из таблицы Product Information. Результат достаточно вернуть в виде словаря.

**Не забывайте про соблюдение PEP-8** — помимо качественно написанного кода важно также документировать функции по стандарту:
* кратко описать, что она делает и для чего нужна;
* какие входные аргументы принимает, какого они типа и что означают по смыслу;
* аналогично описать возвращаемые значения.

*P. S. Состав, количество аргументов функции и тип возвращаемого значения можете менять как вам удобно. То, что написано ниже в шаблоне — лишь пример.*

In [253]:
def get_book_data(
    book_url: str,
    class_table: str = "table table-striped",
    must_have_data: list = None
) -> dict:
    """
    Извлекает данные о книге из HTML-таблицы на веб-странице.

    Args:
        book_url (str): URL-адрес страницы с информацией о книге
        class_table (str): CSS-класс таблицы (по умолчанию "table table-striped")
        must_have_data (list): Список обязательных полей для проверки

    Returns:
        dict: Словарь с данными о книге

    Raises:
        ValueError: При некорректных входных параметрах
        requests.exceptions.RequestException: При ошибках HTTP-запроса
        Exception: При ошибках парсинга HTML-контента
    """

    # Кастомные исключения
    class BookPageParseError(Exception):
        """Исключение для ошибок парсинга страницы книги."""
        pass

    class BookDataNotFoundError(Exception):
        """Исключение когда не найдены критические данные книги."""
        pass

    # Инициализация must_have_data по умолчанию
    if must_have_data is None:
        must_have_data = []

    parsed = urlparse(book_url)
    base_url = f"{parsed.scheme}://{parsed.netloc}/"
    
    try:
        if not isinstance(book_url, str):
            raise ValueError(f"URL должен быть строкой, получен {type(book_url)}")

        # Выполнение HTTP-запроса
        try:
            response = requests.get(book_url)
            response.raise_for_status()
        except requests.exceptions.RequestException as e:
            raise requests.exceptions.RequestException(
                f"Ошибка выполнения HTTP-запроса {response.status_code}: {e}"
            )

        # Парсинг
        try:
            soup = BeautifulSoup(response.text, 'html.parser')
        except Exception as e:
            raise BookPageParseError(f"Ошибка парсинга HTML: {e}")

        data = {}

        data['title'] = soup.find('h1').text
        data['price'] = soup.find('p', class_='price_color').text
        data['description'] = soup.find('meta', attrs={'name': 'description'})['content']

        rating_element = soup.find('p', class_='star-rating')
        data['rating'] = rating_element['class'][1] if rating_element else None

        availability = soup.find('p', class_='instock')
        data['availability'] = availability.text.strip() if availability else None

        breadcrumb = soup.find('ul', class_='breadcrumb')
        category_links = breadcrumb.find_all('li')
        data['category'] = category_links[2].text.strip() if len(category_links) > 2 else None

        table = soup.find(class_=class_table)

        headers = map(lambda x: x.get_text(strip=True), table.find_all('th'))
        data_cells = map(lambda x: x.get_text(strip=True), table.find_all('td'))

        data['product_info'] = dict(zip(headers, data_cells))

        image = soup.find('div', class_='item active').find('img')
        data['image_url'] = base_url + image['src'].replace('../..', '') if image else None

        none_data = set(must_have_data) & {key for key, value in data.items() if value is None}
        if none_data:
            raise BookDataNotFoundError(f"Обязательные поля не заполнены: {', '.join(none_data)}")

        return data

    except (ValueError, requests.exceptions.RequestException,
            BookPageParseError, BookDataNotFoundError):
        raise

    except Exception as e:
        raise BookPageParseError(f"Непредвиденная ошибка при обработке страницы: {e}")
    # КОНЕЦ ВАШЕГО РЕШЕНИЯ

In [254]:
# Используйте для самопроверки
book_url = 'http://books.toscrape.com/catalogue/a-light-in-the-attic_1000/index.html'
get_book_data(book_url)

{'title': 'A Light in the Attic',
 'price': 'Â£51.77',
 'description': "\n    It's hard to imagine a world without A Light in the Attic. This now-classic collection of poetry and drawings from Shel Silverstein celebrates its 20th anniversary with this special edition. Silverstein's humorous and creative verse can amuse the dowdiest of readers. Lemon-faced adults and fidgety kids sit still and read these rhythmic words and laugh and smile and love th It's hard to imagine a world without A Light in the Attic. This now-classic collection of poetry and drawings from Shel Silverstein celebrates its 20th anniversary with this special edition. Silverstein's humorous and creative verse can amuse the dowdiest of readers. Lemon-faced adults and fidgety kids sit still and read these rhythmic words and laugh and smile and love that Silverstein. Need proof of his genius? RockabyeRockabye baby, in the treetopDon't you know a treetopIs no safe place to rock?And who put you up there,And your cradle, t

## Задание 2. Сбор данных обо всех книгах (20 баллов)

Создайте функцию `scrape_books`, которая будет проходиться по всем страницам из каталога (вида `http://books.toscrape.com/catalogue/page-{N}.html`) и осуществлять парсинг всех страниц в цикле, используя ранее написанную `get_book_data`.

Добавьте аргумент-флаг, который будет отвечать за сохранение результата в файл: если он будет равен `True`, то информация сохранится в ту же папку в файл `books_data.txt`; иначе шаг сохранения будет пропущен.

**Также не забывайте про соблюдение PEP-8**

In [255]:
def scrape_books(
    is_save=True, 
    base_url="http://books.toscrape.com/catalogue/page-{}.html", 
    page_num=1, 
    class_list="image_container",
    file_name='books_data.txt',
    format_data=str,
    stop=float('inf'),
    func_get=get_book_data
): 

    """
    Парсит данные о книгах с сайта <base_url>.
    
    Функция проходит по указанному количеству страниц каталога, извлекает 
    ссылки на книги и собирает информацию о каждой книге.
    
    Args:
        is_save (bool): Сохранять ли данные в файл. По умолчанию True
        base_url (str): Шаблон URL для pagination. По умолчанию URL books.toscrape.com
        page_num (int): Номер страницы для начала парсинга. По умолчанию 1
        class_list (str): CSS класс для поиска контейнеров с книгами. По умолчанию "image_container"
        file_name (str): Имя файла для сохранения данных. По умолчанию 'books_data.txt'
        format_data (callable): Функция для форматирования данных перед сохранением. По умолчанию str
        stop (int): Номер страницы на которой остановиться. По умолчанию 2
        
    Returns:
        list: Список словарей с информацией о книгах
        
    Examples:
        >>> # Парсинг первых 2 страниц
        >>> books = scrape_books(stop=2)
        >>> # Парсинг с кастомным форматированием
    """
    
    # НАЧАЛО ВАШЕГО РЕШЕНИЯ
    
    data = {}
    book_num = 1
    
    while page_num < stop:
        url = base_url.format(page_num)
        response = requests.get(url)
        
        if response.status_code != 200:
            break  # Закончились страницы

        try:
            soup = BeautifulSoup(response.text, 'html.parser').find_all(class_=class_list)
        
            for link in soup:
                full_url = "http://books.toscrape.com/catalogue/"+link.find('a', href=True)['href']
                data[book_num] = func_get(full_url)  
                book_num += 1
            
            page_num += 1
        
            time.sleep(0.5)
                  
            if is_save:        
                with open (file_name, 'w', encoding='utf-8') as f:
                    for book in data.items():
                        f.write(format_data(book)+'\n')
                        
        except requests.exceptions.RequestException as e:
            print(f"Ошибка запроса: {e}")
            return {}
        except Exception as e:
            print(f"Ошибка парсинга: {e}")
            return {}     
        
    return data
    # КОНЕЦ ВАШЕГО РЕШЕНИЯ

In [256]:
# Проверка работоспособности функции
res = scrape_books(is_save=True, stop=2, func_get=get_book_data) # Допишите ваши аргументы
print(type(res), len(res)) # и проверки

<class 'dict'> 20


## Задание 3. Настройка регулярной выгрузки (10 баллов)

Настройте автоматический запуск функции сбора данных каждый день в 19:00.
Для автоматизации используйте библиотеку `schedule`. Функция должна запускаться в указанное время и сохранять обновленные данные в текстовый файл.



Бесконечный цикл должен обеспечивать постоянное ожидание времени для запуска задачи и выполнять ее по расписанию. Однако чтобы не перегружать систему, стоит подумать о том, чтобы выполнять проверку нужного времени не постоянно, а раз в какой-то промежуток. В этом вам может помочь `time.sleep(...)`.

Проверьте работоспособность кода локально на любом времени чч:мм.



In [ ]:
# НАЧАЛО ВАШЕГО РЕШЕНИЯ

schedule.every().day.at("19:00", "Europe/Moscow").do(scrape_books(is_save=True, func_get=get_book_data))

while True:
    schedule.run_pending()
    time.sleep(1)
    
# КОНЕЦ ВАШЕГО РЕШЕНИЯ

## Задание 4. Написание автотестов (15 баллов)

Создайте минимум три автотеста для ключевых функций парсинга — например, `get_book_data` и `scrape_books`. Идеи проверок (можете использовать свои):

* данные о книге возвращаются в виде словаря с нужными ключами;
* список ссылок или количество собранных книг соответствует ожиданиям;
* значения отдельных полей (например, `title`) корректны.

Оформите тесты в отдельном скрипте `tests/test_scraper.py`, используйте библиотеку `pytest`. Убедитесь, что тесты проходят успешно при запуске из терминала командой `pytest`.

Также выведите результат их выполнения в ячейке ниже.

**Не забывайте про соблюдение PEP-8**


In [ ]:
# Ячейка для демонстрации работоспособности
# Сам код напишите в отдельном скрипте
! pytest test/test_scraper.py

## Задание 5. Оформление проекта на GitHub и работа с Git (35 баллов)

В этом задании нужно воспользоваться системой контроля версий Git и платформой GitHub для хранения и управления своим проектом. **Ссылку на свой репозиторий пришлите в форме для сдачи ответа.**

### Пошаговая инструкция и задания

**1. Установите Git на свой компьютер.**

* Для Windows: [скачайте установщик](https://git-scm.com/downloads) и выполните установку.
* Для macOS:

  ```
  brew install git
  ```
* Для Linux:

  ```
  sudo apt update
  sudo apt install git
  ```

**2. Настройте имя пользователя и email.**

Это нужно для подписи ваших коммитов, сделайте в терминале через `git config ...`.

**3. Создайте аккаунт на GitHub**, если у вас его еще нет:
[https://github.com](https://github.com)

**4. Создайте новый репозиторий на GitHub:**

* Найдите кнопку **New repository**.
* Укажите название, краткое описание, выберите тип **Public** (чтобы мы могли проверить ДЗ).
* Не ставьте галочку Initialize this repository with a README.

**5. Создайте локальную папку с проектом.** Можно в терминале, можно через UI, это не имеет значения.

**6. Инициализируйте Git в этой папке.** Здесь уже придется воспользоваться некоторой командой в терминале.

**7. Привяжите локальный репозиторий к удаленному на GitHub.**

**8. Создайте ветку разработки.** По умолчанию вы будете находиться в ветке `main`, создайте и переключитесь на ветку `hw-books-parser`.

**9. Добавьте в проект следующие файлы и папки:**

* `scraper.py` — ваш основной скрипт для сбора данных.
* `README.md` — файл с кратким описанием проекта:

  * цель;
  * инструкции по запуску;
  * список используемых библиотек.
* `requirements.txt` — файл со списком зависимостей, необходимых для проекта (не присылайте все из глобального окружения, создайте изолированную виртуальную среду, добавьте в нее все нужное для проекта и получите список библиотек через `pip freeze`).
* `artifacts/` — папка с результатами парсинга (`books_data.txt` — полностью или его часть, если весь не поместится на GitHub).
* `notebooks/` — папка с заполненным ноутбуком `HW_03_python_ds_2025.ipynb` и запущенными ячейками с выводами на экран.
* `tests/` — папка с тестами на `pytest`, оформите их в формате скрипта(-ов) с расширением `.py`.
* `.gitignore` — стандартный файл, который позволит исключить временные файлы при добавлении в отслеживаемые (например, `__pycache__/`, `.DS_Store`, `*.pyc`, `venv/` и др.).


**10. Сделайте коммит.**

**11. Отправьте свою ветку на GitHub.**

**12. Создайте Pull Request:**

* Перейдите в репозиторий на GitHub.
* Нажмите кнопку **Compare & pull request**.
* Укажите, что было добавлено, и нажмите **Create pull request**.

**13. Выполните слияние Pull Request:**

* Убедитесь, что нет конфликтов.
* Нажмите **Merge pull request**, затем **Confirm merge**.

**14. Скачайте изменения из основной ветки локально.**



### Требования к итоговому репозиторию

* Файл `scraper.py` с рабочим кодом парсера.
* `README.md` с описанием проекта и инструкцией по запуску.
* Папка `artifacts/` с результатом сбора данных (`.txt` файл).
* Папка `tests/` с тестами на `pytest`.
* Папка `notebooks/` с заполненным ноутбуком `HW_03_python_ds_2025.ipynb`.
* Pull Request с комментарием из ветки `hw-books-parser` в ветку `main`.
* Примерная структура:

  ```
  books_scraper/
  ├── artifacts/
  │   └── books_data.txt
  ├── notebooks/
  │   └── HW_03_python_ds_2025.ipynb
  ├── scraper.py
  ├── README.md
  ├── tests/
  │   └── test_scraper.py
  ├── .gitignore
  └── requirements.txt
  ```